In [5]:
import pandas as pd

df = pd.read_pickle(r'data\data_df.pkl')
df.head()

,OriginalText,CleanedText,WordList,Sentyment,Lubienie,Wstręt,Złość,Strach,Zaskoczenie,Oczekiwanie,Radość,Smutek
AF1.txt,Wczoraj rano przytrafiło mi się coś miłego. Pr...,wczoraj rano przytrafiło mi się coś miłego prz...,"[wczoraj, rano, przytrafiło, mi, się, coś, mił...",5.0,2.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0
AF10.txt,"Pan Dussel, człowiek, o którym zawsze mówiono,...",pan dussel człowiek o którym zawsze mówiono że...,"[pan, dussel, człowiek, o, którym, zawsze, mów...",1.0,0.0,1.0,4.0,0.0,0.0,1.0,0.0,2.0
AF11.txt,Jakbym w ciągu dnia nie słyszała wystarczająco...,jakbym w ciągu dnia nie słyszała wystarczająco...,"[jakbym, w, ciągu, dnia, nie, słyszała, wystar...",1.0,0.0,1.0,3.0,1.0,0.0,1.0,0.0,2.0
AF12.txt,"Pienię się z wściekłości, a nie wolno mi tego ...",pienię się z wściekłości a nie wolno mi tego o...,"[pienię, się, z, wściekłości, a, nie, wolno, m...",1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0
AF13.txt,"Wszyscy uważają, że jestem przesadna, gdy mówi...",wszyscy uważają że jestem przesadna gdy mówię ...,"[wszyscy, uważają, że, jestem, przesadna, gdy,...",1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,3.0


In [6]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
X = df['CleanedText']

ylabels = df['Smutek'] # the labels, or answers, we want to test against

vector = CountVectorizer()
X_vec = vector.fit_transform(X).toarray()
X_train, X_test, y_train, y_test = train_test_split(X_vec, ylabels, test_size=0.3)

In [7]:
from sklearn import svm
regr = svm.SVC()
regr.fit(X_train,y_train)


SVC()

In [10]:
print(regr.predict(X_test))
print(y_test.values)


[0. 0. 0. 2. 2. 0. 0. 0. 0. 2. 0. 2. 2. 0. 2. 0. 0. 0. 2. 0. 0. 0. 2. 0.
 0. 0. 0. 2. 0. 2. 0.]
[3. 0. 3. 3. 3. 3. 0. 3. 2. 1. 0. 0. 2. 0. 2. 1. 2. 2. 0. 2. 0. 4. 3. 2.
 1. 0. 2. 0. 3. 1. 3.]
